In [1]:
import tushare as ts

In [4]:
from sqlalchemy import create_engine

tushare_db = create_engine('postgresql://earthson@localhost/trading_tushare', pool_size=32, max_overflow=0)

In [1]:
def fetch_stock_basics():
    df = ts.get_stock_basics()
    df.to_sql('stock_basics', tushare_db, if_exists="replace")

def fetch_tick(stock, date):
    df = ts.get_tick_data(stock, date=date)
    df['stock'] = stock
    df['date'] = date
    ans = df.set_index(['stock', 'date'])
    try:
        tushare_db.execute("""delete from tick_data where "stock"='%s' AND "date"='%s' """ % (stock, date))
    except:
        pass
    ans.to_sql('tick_data', tushare_db, if_exists="append")

In [2]:
#History data forward answer authority
def fetch_history_faa(stock, start, end):
    df = ts.get_h_data(stock, autype='hfq', start=start, end=end)
    df['stock'] = stock
    ans = df.set_index(['stock','date'])
    try:
        tushare_db.execute("""delete from history_faa where "stock"='%s' AND "date">='%s' AND "date"<='%s'""" % (stock, start, end))
    except:
        pass
    ans.to_sql('history_faa', tushare_db, if_exists="append")

In [2]:
df = ts.get_tick_data('002218', date='2008-03-24')
df['stock'] = '002218'

In [3]:
df2 = ts.get_tick_data('002218', date='2008-03-25')

In [5]:
df2['stock'] = '002218'

In [7]:
len(df) + len(df2)

2758

In [8]:
import pandas as pd

In [11]:
pd.concat([df, df2])

,time,price,change,volume,amount,type,stock
0,15:00:34,36.00,-0.1,382,1377000,卖盘,002218
1,14:56:58,36.10,-0.18,18,64980,卖盘,002218
2,14:56:48,36.28,-0.01,2,9070,买盘,002218
3,14:56:43,36.29,--,2,7258,买盘,002218
4,14:56:33,36.29,--,11,39919,买盘,002218
5,14:56:23,36.29,-0.01,23,83467,中性盘,002218
6,14:56:13,36.30,--,198,718740,买盘,002218
7,14:56:03,36.30,0.01,14,50819,卖盘,002218
8,14:55:58,36.29,-0.01,17,61693,卖盘,002218
9,14:55:53,36.30,--,41,148830,买盘,002218


In [20]:
from datetime import datetime
time_fmt = '%Y-%m-%d %H:%M:%S'

In [5]:
import pandas as pd

In [84]:
df['time'] = ('2008-03-24 ' + df['time']).map(lambda x: pd.Timestamp(datetime.strptime(x, time_fmt), tz='Asia/Shanghai').strftime(format="%Y-%m-%d %H:%M:%S%z"))

In [88]:
bb = df.set_index(['stock', 'time'])

In [89]:
from sqlalchemy import create_engine
from sqlalchemy import TIMESTAMP
tushare_db = create_engine("postgresql+psycopg2://earthson@localhost/trading_tushare", pool_size=16, max_overflow=0)

In [91]:

bb.to_sql('tick_data', tushare_db, dtype={'time': TIMESTAMP(timezone=True)})

In [95]:
hdf = ts.get_h_data('600654', autype='hfq', start='1991-09-01', end='1992-05-14').reset_index()

[Getting data:]###